### Week of 9/13 - 9/20

pip install:
scipy
seaborn
pandas
tables

Relevant function from comb-building, addition/modification for foraging below

In [41]:
def interaction_events(dfTraj,beeID,minTime,maxTime,distance_thresh=10):
    """
    Returns dataFrame listing interaction events during which the bee with ID beeID is nearby another bee, within
    the time window between minTime and maxTime.  Here "nearby" means within a distance threshold and on the same 
    side of the hive.

    dfTraj                   : Trajectory dataframe containing framenumbers, (x,y) coordinates, and camera information.
    minTime                  : Minimum time to scan for interaction events, as a datetime object
    maxTime                  : Maximum time to scan for interaction events, as a datetime object
    distance_thresh (10)     : Distance in pixels within which two bees must be imaged in the same frame to count as an interaction
    """
    # select trajectory data from all bees from the desired time window
    minFrame = datetime_to_framenum(minTime)
    maxFrame = datetime_to_framenum(maxTime)
    dfTraj_selected = dfTraj[(minFrame < dfTraj['framenum']) & (dfTraj['framenum'] < maxFrame)]

    # select trajectory data from the selected bee
    dfTraj_selected_bee = dfTraj_selected[dfTraj_selected['uid']==beeID]
    
    # loop over time looking for interaction events
    interaction_data = []
    for frame in dfTraj_selected_bee['framenum'].unique():
        all_bee_frame_data = dfTraj_selected[dfTraj_selected['framenum']==frame]
        
        # get selected bee location for this time frame
        selected_bee_frame_data = all_bee_frame_data[all_bee_frame_data['uid']==beeID]
        beex = selected_bee_frame_data['x'].iloc[0],
        beey = selected_bee_frame_data['y'].iloc[0]
        beecamera = selected_bee_frame_data['camera'].iloc[0]

        # find distances from all other bees seen by the same camera (same side of hive) at this time frame
        all_bee_frame_data_same_side = all_bee_frame_data[all_bee_frame_data['camera']==beecamera]
        distances = pd.Series(np.linalg.norm([all_bee_frame_data_same_side['x']-beex,
                                              all_bee_frame_data_same_side['y']-beey],axis=0),
                              index=all_bee_frame_data_same_side['uid'])

        # append interaction data
        for other_bee in distances[distances < distance_thresh].index:
            if other_bee != beeID:
                other_bee_frame_data = all_bee_frame_data_same_side[all_bee_frame_data_same_side['uid']==other_bee]
                assert(len(other_bee_frame_data)==1) # should only have one timepoint for one bee in one frame
                interaction_data.append([frame,
                                         other_bee,
                                         other_bee_frame_data['x'].iloc[0],
                                         other_bee_frame_data['y'].iloc[0],
                                         other_bee_frame_data['camera'].iloc[0],
                                         distances[other_bee]])
    
    return pd.DataFrame(interaction_data,columns=['framenum','uid','x','y','camera','distance (pixels)'])

In [ ]:
def number_of_interactions(dfTraj, beeID, minTime, maxTime, distance_thresh):
    